In [33]:
import numpy as n
import numpy as np
import pandas as pd


In [3]:
MILANO = pd.read_csv('../Dataset/Comune-di-Milano-Pubblici-esercizi(in)-2.csv',sep=';',encoding='utf-8')

In [28]:
settore_col = "Settore storico pe"
tipo_col    = "þÿTipo esercizio storico pe"

df = MILANO.copy()

# 1) Normalizzo il testo del settore per cercare le parole chiave in modo robusto
df["settore_norm"] = df[settore_col].astype(str).str.upper().fillna("")

# 2) Definisco i pattern per ogni macro-categoria -------------------------

# 1) BAR
patterns_bar = [
    "BAR CAFFE",                  # bar caffe
    "CAFF",                   # bar caffÿ, bar-caffè e simili
    "BAR GASTRONOM",              # bar gastronomici
    "BIRRERIA",                   # birreria
    "SALE DA BALLO",              # sale da ballo, locali notturni
    "BAR",
    "DISCO",
    "LOCALI NOTTURNI",
    "SPACCIO BEVANDE ANALCOLICHE",
    "GIOC",
    "SOMMINISTRAZIONE",
    "WINE",                       # wine,birr.,pub enot.,caff.,the
    "PUB"
]

# 2) PIZZERIA
patterns_piz = [
    "PIZZERIA",
    "PIZZERIE E SIMILI"
]

# 3) RISTORANTE
patterns_rist = [
    "RISTORA", #inclueds ristorante and ristorazione
    "OSTERIA",
    "CUCINA",
    "TRATTORIA"
    # copre anche "RISTORANTE, TRATTORIA, OSTERIA", "TRTTORIA" ecc. per match parziale
]

# 4) GASTRONOMIA
patterns_gast = [
    "GENERE MERCEOL",                # genere merceol.autorizz.sanit.
    "PRODOTTI DI GASTRONOMIA",      # prodotti di gastronomia
    "PROD DI GASTRO",               # varianti abbreviate
    "TAVOLA FREDDA",
    "CIBI COTTI",
    "CIBI COTTI PRECONFEZIONATI",
    "MENSA",
    "TAVOLA CALDA",
    "TAV.CALDE,SELF SERVICE,FAST F", # tav.calde,self service,fast f.
    "SELF SERVICE",
    "FAST F"
]

# 5) GELATERIA
patterns_gel = [
    "BAR PASTIC",   # bar pasticc.gelat.crem.creper.
    "GELATERIA"
]



# 3) Creo i mask per ogni macro-categoria ---------------------------------

def build_mask(patterns):
    """Ritorna una mask booleana True se settore_norm contiene almeno uno dei pattern."""
    regex = "|".join(patterns)
    return df["settore_norm"].str.contains(regex, na=False)

mask_bar   = build_mask(patterns_bar)
mask_piz   = build_mask(patterns_piz)
mask_rist  = build_mask(patterns_rist)
mask_gast  = build_mask(patterns_gast)
mask_gel   = build_mask(patterns_gel)

# 4) Assegno le macro-categorie con la GERARCHIA voluta -------------------
#    RISTORANTE > PIZZERIA > BAR > GASTRONOMIA > GELATERIA
#    Quindi metto prima le categorie a priorità più bassa, e per ultime quelle a priorità più alta.

df["Tipo_macro"] = "ALTRO"   # default

# Priorità più bassa: GELATERIA
df.loc[mask_gel,  "Tipo_macro"] = "GELATERIA"

# Poi GASTRONOMIA
df.loc[mask_gast, "Tipo_macro"] = "GASTRONOMIA"

# Poi BAR
df.loc[mask_bar,  "Tipo_macro"] = "BAR"

# Poi PIZZERIA
df.loc[mask_piz,  "Tipo_macro"] = "PIZZERIA"

# Infine RISTORANTE (priorità MASSIMA: sovrascrive PIZZERIA/BAR/GASTRONOMIA/GELATERIA)
df.loc[mask_rist, "Tipo_macro"] = "RISTORANTE"

# 5) Copio il risultato su MILANO -----------------------------------------

# Creo una nuova colonna con la macro-categoria
MILANO["Tipo esercizio macro pe"] = df["Tipo_macro"]

# Se vuoi DAVVERO riscrivere la colonna storica (io terrei l'originale):
# MILANO[tipo_col] = MILANO["Tipo esercizio macro pe"]

# Controllo rapido delle frequenze
MILANO.groupby("Tipo esercizio macro pe").count()

,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione
Tipo esercizio macro pe,,,,,,,,,,,,,
ALTRO,6,2,21,21,21,21,21,21,6,6,6,0,17
BAR,3049,1961,3654,3654,3654,3577,3654,3654,2922,3012,3000,3654,3629
GASTRONOMIA,22,15,246,246,246,239,246,246,23,23,22,246,217
GELATERIA,3,0,3,3,3,3,3,3,2,2,2,3,3
PIZZERIA,181,109,246,246,246,237,246,246,172,177,175,246,245
RISTORANTE,2290,1407,2734,2734,2734,2671,2734,2734,2208,2287,2275,2734,2714


In [13]:
idx_mismatch_zd = MILANO["Tipo esercizio macro pe"] == "ALTRO"
idx_mismatch_zd

0       False
1       False
2       False
3       False
4       False
        ...  
6899    False
6900    False
6901    False
6902    False
6903    False
Name: Tipo esercizio macro pe, Length: 6904, dtype: bool

In [35]:
ALTRO = MILANO[MILANO["Tipo esercizio macro pe"] == "ALTRO"]

ALTRO[""]

ValueError: Cannot set a DataFrame with multiple columns to the single column tipo es

In [39]:
MILANO["Tipo esercizio macro pe"].isnull().sum()

0

In [41]:
MILANO["Insegna"] = MILANO["Insegna"].fillna("Unknown")
MILANO

,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione,Tipo esercizio macro pe
0,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,NaN,NaN,NaN,"Ristorante, trattoria, osteria;Genere Merceol....",83.0,RISTORANTE
1,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,26.0,BAR
2,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,58.0,BAR
3,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,NaN,NaN,NaN,"BAR CAFFÿý E SIMILI;Ristorante, trattoria, ost...",101.0,RISTORANTE
4,NaN,Unknown,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,NaN,NaN,NaN,Bar gastronomici e simili,51.0,BAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6899,"wine,birr.,pub enot.,caff.,the",bar cherry,VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...,VLE,DORIA ANDREA,12,2230,2,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",59.0,BAR
6900,"wine,birr.,pub enot.,caff.,the",la balusa,VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...,VIA,GARIGLIANO,5,1134,9,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",40.0,BAR
6901,"wine,birr.,pub enot.,caff.,the",la champagnerie sas,VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...,VIA,SOTTOCORNO PASQUALE,4,3152,4,solo somministrazione,somministrazione,misto,BAR CAFFÿý E SIMILI;Bar gastronomici e simili,53.0,BAR
6902,"wine,birr.,pub enot.,caff.,the",old rooster,VIA CASTROVILLARI N. 23 ; isolato:150; accesso...,VIA,CASTROVILLARI,23,6299,7,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",43.0,BAR


In [45]:
MILANO["Superficie somministrazione"] = pd.to_numeric(
    MILANO["Superficie somministrazione"],
    errors="coerce"
)

In [47]:
superficie_mean = MILANO["Superficie somministrazione"].mean()

MILANO["Superficie somministrazione"] = MILANO["Superficie somministrazione"].fillna(superficie_mean)
MILANO

,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione,Tipo esercizio macro pe
0,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,NaN,NaN,NaN,"Ristorante, trattoria, osteria;Genere Merceol....",83.0,RISTORANTE
1,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,26.0,BAR
2,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,58.0,BAR
3,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,NaN,NaN,NaN,"BAR CAFFÿý E SIMILI;Ristorante, trattoria, ost...",101.0,RISTORANTE
4,NaN,Unknown,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,NaN,NaN,NaN,Bar gastronomici e simili,51.0,BAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6899,"wine,birr.,pub enot.,caff.,the",bar cherry,VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...,VLE,DORIA ANDREA,12,2230,2,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",59.0,BAR
6900,"wine,birr.,pub enot.,caff.,the",la balusa,VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...,VIA,GARIGLIANO,5,1134,9,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",40.0,BAR
6901,"wine,birr.,pub enot.,caff.,the",la champagnerie sas,VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...,VIA,SOTTOCORNO PASQUALE,4,3152,4,solo somministrazione,somministrazione,misto,BAR CAFFÿý E SIMILI;Bar gastronomici e simili,53.0,BAR
6902,"wine,birr.,pub enot.,caff.,the",old rooster,VIA CASTROVILLARI N. 23 ; isolato:150; accesso...,VIA,CASTROVILLARI,23,6299,7,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",43.0,BAR
